In [18]:
import gym
import random
import numpy as np
import time
from gym.envs.registration import register
from IPython.display import clear_output

In [19]:
# env_name = 'CartPole-v1'
# env_name = 'MountainCarContinuous-v0'
# env_name = 'Acrobot-v1'
# env_name = 'Pendulum-v0'
try:
    register(
    id='FrozenLakeNoSlip-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery':False},
    max_episode_steps=100,
    reward_threshold=0.78, # optimum = .8196
    )
except:
    pass


env_name = "FrozenLake-v0"
env = gym.make(env_name)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
type(env.action_space)

Observation space: Discrete(16)
Action space: Discrete(4)


gym.spaces.discrete.Discrete

In [24]:
class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("Action size:", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("Action range:", self.action_low, self.action_high)
        
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                       self.action_high,
                                       self.action_shape)
        return action

In [25]:
class QAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
    def build_model(self):
        self.q_table = 1e-4*np.random.random([self.state_size, self.action_size])
        
    def get_action(self, state):
        q_state = self.q_table[state]
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_next = self.q_table[next_state]
        q_next = np.zeros([self.action_size]) if done else q_next
        q_target = reward + self.discount_rate * np.max(q_next)
        
        q_update = q_target - self.q_table[state,action]
        self.q_table[state,action] += self.learning_rate * q_update
        
        if done:
            self.eps = self.eps * 0.99
        
agent = QAgent(env)

Action size: 4
State size: 16


In [29]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()
        print(agent.q_table)
        time.sleep(0.05)
        clear_output(wait=True)

s: 5 a: 0
Episode: 99, Total reward: 31.0, eps: 0.017950553275045134
  (Left)
SFFF
FHFH
FFFH
HFFG
[[2.30522788e-03 9.41776644e-05 1.29172469e-04 8.90383845e-05]
 [5.17137334e-05 4.40638884e-05 5.21212027e-05 1.41981964e-04]
 [3.91600613e-05 5.64920714e-05 1.58647982e-05 7.40303348e-05]
 [6.41194441e-05 6.16986012e-05 6.45882066e-05 3.95497220e-05]
 [4.21361068e-03 1.00362069e-04 7.62192640e-05 3.56175338e-05]
 [1.10193587e-05 6.55660891e-05 4.17423660e-05 8.28320139e-05]
 [4.99387089e-03 4.06822913e-06 1.80630181e-05 5.38684454e-05]
 [8.90535003e-06 8.11059706e-05 4.34574916e-05 2.44603708e-05]
 [8.63430932e-05 3.99110976e-04 1.17881162e-04 9.54201439e-03]
 [4.01082659e-04 2.70148666e-02 8.91535274e-04 9.99290316e-04]
 [7.05860940e-02 3.01710020e-03 1.70104264e-04 5.34103909e-05]
 [3.91375511e-05 1.28638552e-05 1.04236246e-05 5.59791343e-05]
 [8.58869703e-05 9.61173841e-05 4.28091149e-06 4.80429836e-05]
 [5.83200496e-05 9.23532911e-04 1.71621735e-04 3.88003268e-02]
 [2.24998041e-04 2.0